<a href="https://colab.research.google.com/github/girishpalange/DS_Projects/blob/main/ShellHK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.optimize import minimize

In [ ]:
dmh = pd.read_csv( "Demand_History.csv")
einfra = pd.read_csv( "/content/existing_EV_infrastructure_2018.csv")

In [ ]:
# shape of dmh is (4096 rows, 12 columns)
# column hearders are ['demand_point_index', 'x_coordinate', 'y_coordinate', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018']
# except 1 column all column datatype is float 64, 1 column is integer 64
#dmh.hist(bins=50, figsize=(20, 15))
#plt.show()

# Get the Demand data(column 3 onwards) for each of the points from 0-4095
x = dmh.iloc[:, 3:12]

# Transpose Demand data for all demand points
dmh_T = x.T

# Demand matrix 0 subset of transpose data
t = dmh_T.iloc[:, 0]
# Convert t to dataframe
t = t.to_frame()
c = "Demand_" + str(t.columns.values)
# Convert index to column data
t.reset_index(inplace=True)
# Rename column names - 1) Year 2) Demand_[0]
t = t.rename(columns={'index': 'Year',
                      0: c})


In [ ]:
# shape of dmh is (100 rows, 6 columns)
# column hearders are ['supply_point_index', 'x_coordinate', 'y_coordinate', 'total_parking_slots', 'existing_num_SCS', 'existing_num_FCS']
# dtypes [ int64, float64, float64, int64, int64, int64]

# Calculate total Charging Capacity and add it as a column to einfra dataframe
def cc(row):
    return ((row['existing_num_SCS']) * 200 + (row['existing_num_FCS']) * 400)

einfra['CC'] = einfra.apply(lambda row: cc(row), axis=1)
#einfra.to_csv(r"existing_EV_infrastructure_2018_Updated.csv")

In [ ]:
# Function to calculate distance between the demand points and supply points for 2018
def d(x1, y1, row):
    m1 = np.array([x1,y1])
    m2 = np.array([row['x_coordinate'], row['y_coordinate']])
    return np.linalg.norm(m2-m1)

# Loop to get the Distance columns between supply and demand
for i in einfra.index:
    dst = "DST_" + str(i)
    x1 = einfra['x_coordinate'].values[i]
    y1 = einfra['y_coordinate'].values[i]
    dmh[dst] = dmh.apply(lambda row: d(x1, y1, row), axis=1)

In [ ]:
# Create columns for Demand supplied by each supply point and intialize the value = ['2018'] / 100
for i in einfra.index:
    dm = "DM_" + str(i)
    dmh[dm] = dmh.apply(lambda x: x['2018'] / 100, axis=1)

In [ ]:
for i in (dmh.index):
  dm1 = dmh.iloc[i:i+1, 12:112].values
  dm2 = dmh.iloc[i:i+1, 112:213].values
  
  def obj(x):
    dm2 = x.T
    return np.dot(dm1,dm2)
  
  dm_2018 = dmh.loc[i, '2018']
  
  sum = 0
  def const1(x):
    sum = x.sum()
    return (dm_2018 - sum)
  
  b = (0.0,320.0)
  b = ((b, ) * 100)

  con1 = {'type': 'eq', 'fun': const1}
  cons = [con1]
  dm2 = dm2.T
  sol = minimize(obj,dm2,method='SLSQP',bounds=b,constraints=cons)

  dmh.iloc[i:i+1, 112:213] = sol.x

In [ ]:
#dmh_T = x.T
#dmh_T = pd.DataFrame(dmh_T)
dmh_T.reset_index(inplace=True)
dmh_T = dmh_T.rename(columns={'index': 'Year'})
#dmh_T.head()

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(dmh_T[0])
dataset = dataset.window(5, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(5))
dataset = dataset.map(lambda window: (window[:-1], window[-1]))
dataset = dataset.shuffle(buffer_size=9)
dataset = dataset.batch(2).prefetch(1)
for x,y in dataset:
  print("x = ", x.numpy())
  print("y = ", y.numpy())

  

x =  [[0.95859331 2.91190058 4.33827422 6.56199463]
 [0.66793169 0.95859331 2.91190058 4.33827422]]
y =  [8.45441738 6.56199463]
x =  [[2.91190058 4.33827422 6.56199463 8.45441738]
 [0.3522418  0.66793169 0.95859331 2.91190058]]
y =  [10.59532421  4.33827422]
x =  [[ 4.33827422  6.56199463  8.45441738 10.59532421]]
y =  [13.11957206]


In [ ]:
def window_dataset(series, window_size, batch_size, shuffle_buffer):
  dataset = tf.data.Dataset.from_tensor_slices(series)
  dataset = dataset.window(window_size+1, shift=1, drop_remainder=True)
  dataset = dataset.flat_map(lambda window: window.batch(window_size+1))
  dataset = dataset.shuffle(shuffle_buffer).map(lambda window: (window[:-1], window[-1]))
  dataset = dataset.batch(batch_size).prefetch(1)
  return dataset


In [ ]:
# Transpose Demand data for all demand points
t = pd.read_csv( "Demand_History.csv")
dmh_T = t.iloc[:, 3:12].T

window_size = 6
batch_size = 1
shuffle_buffer = 9
i = 0

x1 = []
#for i in (dmh.index):
dataset = window_dataset(dmh_T[0], window_size, batch_size, shuffle_buffer)
l0 = tf.keras.layers.Dense(1, input_shape=[window_size])
model = tf.keras.models.Sequential([l0])
model.compile(loss='mse', optimizer=tf.keras.optimizers.SGD(learning_rate=1e-5,momentum=0.9))
model.fit(dataset,epochs=700,verbose=0)
print("Layer weights {}".format(l0.get_weights()))
x = np.array(dmh_T[i].values)
print(model.predict(x[3:9][np.newaxis]))
print(dmh_T[0][3:9])
#x = np.array(dmh_T[i].values)
#D_2019.append(model.predict(x[2:8][np.newaxis]))
#x1.append(model.predict(x[2:8][np.newaxis]))
#D_2020.append(model.predict(x1[3:9][np.newaxis]))

Layer weights [array([[-0.4574314 ],
       [ 0.09213053],
       [ 0.30014214],
       [ 0.9364861 ],
       [ 0.01786096],
       [ 0.6078445 ]], dtype=float32), array([0.22256264], dtype=float32)]
[[17.341133]]
2013     2.911901
2014     4.338274
2015     6.561995
2016     8.454417
2017    10.595324
2018    13.119572
Name: 0, dtype: float64


In [ ]:
D_2019 = []
D_2020 = []

for i in (dmh.index):
  x = np.array(dmh_T[i].values)
  D_2019.append(model.predict(x[3:9][np.newaxis]))
  x1 = np.append(x,model.predict(x[3:9][np.newaxis]))
  D_2020.append(model.predict(x1[4:10][np.newaxis]))

dmh['2019'] = np.squeeze(D_2019)

dmh['2020'] = np.squeeze(D_2020)

print(dmh.head())

   demand_point_index  x_coordinate  y_coordinate      2010      2011  \
0                   0           0.5           0.5  0.352242  0.667932   
1                   1           1.5           0.5  0.325940  0.591964   
2                   2           2.5           0.5  0.373752  0.591890   
3                   3           3.5           0.5  0.420686  0.584055   
4                   4           4.5           0.5  0.475621  0.647940   

       2012      2013      2014      2015      2016  ...         DM_92  \
0  0.958593  2.911901  4.338274  6.561995  8.454417  ...  5.891952e-11   
1  0.862652  2.589068  4.196034  5.745551  8.753195  ...  1.295227e-10   
2  0.969733  2.641432  3.541772  5.469161  8.414627  ...  6.214870e-11   
3  0.906547  2.378577  3.888121  5.846089  9.083868  ...  0.000000e+00   
4  0.981544  2.665400  4.218711  6.776609  8.851107  ...  1.055083e-10   

          DM_93         DM_94         DM_95         DM_96         DM_97  \
0  0.000000e+00  0.000000e+00  0.000000e+

In [ ]:
dm1 = dmh.iloc[0, 12:112].values
dm2 = dmh.iloc[0, 112:213].values
#print(dm1)
#dmh.info

<bound method DataFrame.info of       demand_point_index  x_coordinate  y_coordinate      2010      2011  \
0                      0           0.5           0.5  0.352242  0.667932   
1                      1           1.5           0.5  0.325940  0.591964   
2                      2           2.5           0.5  0.373752  0.591890   
3                      3           3.5           0.5  0.420686  0.584055   
4                      4           4.5           0.5  0.475621  0.647940   
...                  ...           ...           ...       ...       ...   
4091                4091          59.5          63.5  0.171015  0.334565   
4092                4092          60.5          63.5  0.041716  0.061741   
4093                4093          61.5          63.5  0.100895  0.180352   
4094                4094          62.5          63.5  0.155353  0.290825   
4095                4095          63.5          63.5  0.209372  0.340185   

          2012      2013      2014      2015      2016 

In [ ]:
# Create columns for Demand supplied by each supply point and intialize the value = ['2019'] / 100
for i in einfra.index:
    dm = "DM2019_" + str(i)
    dmh[dm] = dmh.apply(lambda x: x['2019'] / 100, axis=1)

In [ ]:
dmh.iloc[i:i+1, 214:216].head()

,DM2019_0,DM2019_1
0,0.151071,0.151071


In [ ]:
for i in (dmh.index):
  dm1 = dmh.iloc[i:i+1, 12:112].values
  dm2 = dmh.iloc[i:i+1, 112:212].values
  
  def obj(x):
    dm2 = x.T
    return np.dot(dm1,dm2)
  
  dm_2019 = dmh.loc[i, '2019']
  
  sum = 0
  def const1(x):
    sum = x.sum()
    return (dm_2019 - sum)
  
  b = (0.0,320.0)
  b = ((b, ) * 100)

  con1 = {'type': 'eq', 'fun': const1}
  cons = [con1]
  dm2 = dm2.T
  sol = minimize(obj,dm2,method='SLSQP',bounds=b,constraints=cons)

  dmh.iloc[i:i+1, 214:314] = sol.x

In [ ]:
# Create columns for Demand supplied by each supply point and intialize the value = ['2019'] / 100
for i in einfra.index:
    dm = "DM2020_" + str(i)
    dmh[dm] = dmh.apply(lambda x: x['2020'] / 100, axis=1)

In [ ]:
dmh.iloc[i:i+1, 314:414].head()

,DM2020_0,DM2020_1,DM2020_2,DM2020_3,DM2020_4,DM2020_5,DM2020_6,DM2020_7,DM2020_8,DM2020_9,...,DM2020_90,DM2020_91,DM2020_92,DM2020_93,DM2020_94,DM2020_95,DM2020_96,DM2020_97,DM2020_98,DM2020_99
99,0.711815,0.711815,0.711815,0.711815,0.711815,0.711815,0.711815,0.711815,0.711815,0.711815,...,0.711815,0.711815,0.711815,0.711815,0.711815,0.711815,0.711815,0.711815,0.711815,0.711815


In [ ]:
for i in (dmh.index):
  dm1 = dmh.iloc[i:i+1, 12:112].values
  dm2 = dmh.iloc[i:i+1, 112:212].values
  
  def obj(x):
    dm2 = x.T
    return np.dot(dm1,dm2)
  
  dm_2020 = dmh.loc[i, '2020']
  
  sum = 0
  def const1(x):
    sum = x.sum()
    return (dm_2020 - sum)
  
  b = (0.0,320.0)
  b = ((b, ) * 100)

  con1 = {'type': 'eq', 'fun': const1}
  cons = [con1]
  dm2 = dm2.T
  sol = minimize(obj,dm2,method='SLSQP',bounds=b,constraints=cons)

  dmh.iloc[i:i+1, 314:414] = sol.x

In [ ]:
dmh.head()

,demand_point_index,x_coordinate,y_coordinate,2010,2011,2012,2013,2014,2015,2016,...,DM2020_90,DM2020_91,DM2020_92,DM2020_93,DM2020_94,DM2020_95,DM2020_96,DM2020_97,DM2020_98,DM2020_99
0,0,0.5,0.5,0.352242,0.667932,0.958593,2.911901,4.338274,6.561995,8.454417,...,0.000000e+00,1.987179e-10,0.000000e+00,3.669353e-10,3.466875e-10,4.583798e-11,0.000000e+00,0.000000e+00,2.491787e-10,0.000000e+00
1,1,1.5,0.5,0.325940,0.591964,0.862652,2.589068,4.196034,5.745551,8.753195,...,3.011909e-10,0.000000e+00,1.196130e-10,0.000000e+00,0.000000e+00,0.000000e+00,4.575532e-11,8.075685e-11,0.000000e+00,3.665538e-10
2,2,2.5,0.5,0.373752,0.591890,0.969733,2.641432,3.541772,5.469161,8.414627,...,1.292269e-10,0.000000e+00,5.470340e-11,0.000000e+00,0.000000e+00,0.000000e+00,2.182382e-11,3.758372e-11,0.000000e+00,1.583726e-10
3,3,3.5,0.5,0.420686,0.584055,0.906547,2.378577,3.888121,5.846089,9.083868,...,0.000000e+00,1.547675e-11,0.000000e+00,2.992373e-11,2.770564e-11,3.108567e-13,0.000000e+00,0.000000e+00,1.790004e-11,0.000000e+00
4,4,4.5,0.5,0.475621,0.647940,0.981544,2.665400,4.218711,6.776609,8.851107,...,0.000000e+00,5.745231e-11,0.000000e+00,1.404978e-10,1.267345e-10,0.000000e+00,0.000000e+00,0.000000e+00,6.801074e-11,0.000000e+00


In [ ]:
#dmh.to_csv(r"Demand_History_Updated.csv")
dmh['DM2019_38'].head()
#d0 = dmh['DM2019_0'].sum(axis=0)

c0 = einfra.iloc[0,6]
s0 = c0 -d0
print(d0)
print(c0)
print(s0)
#d2 = dmh['DM2020_38'].sum(axis=0)
#print(d2)

2484.4246349958244
2200.0
-284.42463499582436


In [ ]:
d1 = []
for i in range(214,314):
  d1.append(dmh.iloc[:,i].sum(axis=0))

einfra['DM2019'] = d1



,supply_point_index,x_coordinate,y_coordinate,total_parking_slots,existing_num_SCS,existing_num_FCS,CC,DM2019
0,0,50.163110,19.412014,23,5,3,2200.0,2484.424635
1,1,37.336451,58.119225,27,4,7,3600.0,4626.502337
2,2,46.709232,57.525650,31,6,14,6800.0,2280.605331
3,3,30.528626,55.379835,26,5,5,3000.0,2184.960430
4,4,51.521781,35.116755,32,11,6,4600.0,644.063492


In [ ]:
d2 = []
for i in range(314,414):
  d2.append(dmh.iloc[:,i].sum(axis=0))

einfra['DM2020'] = d2


In [ ]:
#einfra.head()
#einfra.to_csv(r"existing_EV_infrastructure_2018_Updated.csv")

In [ ]:
dmh = pd.read_csv( "Demand_History_Updated.csv")
einfra = pd.read_csv( "/content/existing_EV_infrastructure_2018_Updated.csv")


In [ ]:
einfra.head()
#einfra.info

,Unnamed: 0,supply_point_index,x_coordinate,y_coordinate,total_parking_slots,existing_num_SCS,existing_num_FCS,CC,DM2019,DM2020
0,0,0,50.163110,19.412014,23,5,3,2200.0,2484.424635,2751.556198
1,1,1,37.336451,58.119225,27,4,7,3600.0,4626.502337,4996.262955
2,2,2,46.709232,57.525650,31,6,14,6800.0,2280.605331,2550.525329
3,3,3,30.528626,55.379835,26,5,5,3000.0,2184.960430,2612.830681
4,4,4,51.521781,35.116755,32,11,6,4600.0,644.063492,779.392823


In [ ]:
i = 0
sc = einfra.iloc[0:1, :].values
fc = einfra.iloc[i:i+1, 8].values 
#fc = fc[0, 6]
#print(sc)
print(fc)

[2484.424635]


In [ ]:
s = []
f = []
for i in (einfra.index):
  x1 = einfra.iloc[i:i+1, :].values  
  
  def obj(x):
    sc = 200 * x[0]
    fc = 400 * x[1]
    C =  sc + 1.5* fc
    return C
  
  sum = 0
  
  def const1(x):
    #print(x[1])
    sc = x[0]
    fc = x[1]
    tc = einfra.iloc[i:i+1, 4].values 
    sum = sc + fc
    return (tc -sum)

  def const2(x):
    #print(einfra.iloc[i:i+1, 8].values )
    sc = x[0]
    fc = x[1]
    c  = einfra.iloc[i:i+1, 8].values 
    C1 = (200 * sc) + (400 * fc)
    return (c - C1 )
  
  sc = einfra.iloc[i:i+1, 5].values 
  fc = einfra.iloc[i:i+1, 6].values 
  tc = einfra.iloc[i:i+1, 4].values 
  #print(tc)
  bsc = (sc,tc)
  bfc = (fc,tc)
  bnds = (bsc,bfc)

  con1 = {'type': 'eq', 'fun': const1}
  con2 = {'type': 'ineq', 'fun': const2}
  cons = [con1,con2]
  x = einfra.iloc[i:i+1, 5:7].values  
  sol = minimize(obj,x,method='SLSQP',bounds=bnds,constraints=cons)
  
  s.append(sol.x[0])
  f.append(sol.x[1])
  #print(i) 
einfra['SC_2019']  = s
einfra['FC_2019']  = f

In [ ]:
s1 = []
f1 = []
for i in (einfra.index):
  x1 = einfra.iloc[i:i+1, :].values  
  
  def obj(x):
    sc = 200 * x[0]
    fc = 400 * x[1]
    C =  sc + 1.5* fc
    return C
  
  sum = 0
  
  def const1(x):
    #print(x[1])
    sc = x[0]
    fc = x[1]
    tc = einfra.iloc[i:i+1, 4].values 
    sum = sc + fc
    return (tc -sum)

  def const2(x):
    #print(einfra.iloc[i:i+1, 8].values )
    sc = x[0]
    fc = x[1]
    c  = einfra.iloc[i:i+1, 9].values 
    C1 = (200 * sc) + (400 * fc)
    return (c - C1 )
  
  sc = einfra.iloc[i:i+1, 5].values 
  fc = einfra.iloc[i:i+1, 6].values 
  tc = einfra.iloc[i:i+1, 4].values 
  #print(tc)
  bsc = (sc,tc)
  bfc = (fc,tc)
  bnds = (bsc,bfc)

  con1 = {'type': 'eq', 'fun': const1}
  con2 = {'type': 'ineq', 'fun': const2}
  cons = [con1,con2]
  x = einfra.iloc[i:i+1, 5:7].values  
  sol = minimize(obj,x,method='SLSQP',bounds=bnds,constraints=cons)
  
  s1.append(sol.x[0])
  f1.append(sol.x[1])
  #print(i) 
einfra['SC_2020']  = s1
einfra['FC_2020']  = f1

In [ ]:
einfra.to_csv(r"existing_EV_infrastructure_2020_Updated.csv")

In [ ]:
import math

In [ ]:
einfra['SC_2019'] = einfra['SC_2019'].apply(np.ceil)
einfra['SC_2020'] = einfra['SC_2020'].apply(np.ceil)

In [ ]:
einfra.to_csv(r"existing_EV_infrastructure_2020_Updated1.csv")